# Add constant force

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import unit
from openmm import app
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

In [3]:
molecular_system = msm.systems['Trp-Cage']['1l2y.h5msm']

In [4]:
topology = msm.convert(molecular_system, to_form='openmm.Topology')
positions = msm.get(molecular_system, element='atom', structure_indices=0, coordinates=True)
positions = msm.pyunitwizard.convert(positions[0], to_form='openmm.unit')

In [5]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [6]:
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [7]:
temperature = 300*unit.kelvin

In [8]:
integration_timestep = 2.0*unit.femtoseconds
saving_interval = 1.00*unit.picoseconds
logging_interval = 100.00*unit.picoseconds
simulation_time = 1000.*unit.picoseconds

saving_steps = int(saving_interval/integration_timestep)
logging_steps = int(logging_interval/integration_timestep)
md_steps = int(simulation_time/integration_timestep)

In [9]:
friction   = 5.0/unit.picoseconds
integrator = mm.LangevinIntegrator(temperature, friction, integration_timestep)

In [10]:
platform = mm.Platform.getPlatformByName('OpenCL')

In [11]:
simulation = app.Simulation(topology, system, integrator, platform)

In [12]:
simulation.context.setPositions(positions)

In [13]:
msm.get(simulation, n_groups=True)

20

In [14]:
msm.info(simulation, element='atom', selection='group_index==0 and atom_type=="C"')

index,id,name,type,group index,group id,group name,group type,component index,chain index,molecule index,molecule type,entity index,entity name
1,2,CA,C,0,1,ASN,amino acid,0,0,0,peptide,0,None
2,3,C,C,0,1,ASN,amino acid,0,0,0,peptide,0,None
4,5,CB,C,0,1,ASN,amino acid,0,0,0,peptide,0,None
5,6,CG,C,0,1,ASN,amino acid,0,0,0,peptide,0,None


In [ ]:
msm.info(simulation, element='atom', selection='group_index==19 and atom_type=="C"')

In [ ]:
msm.thirds.openmm.forces.pin_atoms(simulation, selection='group_index==0 and atom_name=="CA"')

In [ ]:
msm.thirds.openmm.forces.add_constant_force(simulation, selection='group_index==19 and atom_name=="CA"',
                                            force = '[500,0,0] kilojoules/(mole*nanometer)')

In [ ]:
reporter_tqdm = msm.thirds.openmm.reporters.TQDMReporter(logging_steps, md_steps, temperature=False)
simulation.reporters.append(reporter_tqdm)

In [ ]:
reporter_trajectory_dict = msm.thirds.openmm.reporters.TrajectoryDictReporter(saving_steps, time=True, coordinates=True)
simulation.reporters.append(reporter_trajectory_dict)

In [ ]:
simulation.step(md_steps)

In [ ]:
trajectory_dict = reporter_trajectory_dict.finalize()

In [ ]:
plt.plot(trajectory_dict['time'], trajectory_dict['coordinates'][:,0,2])
plt.axhline(y=1.25, color='red', linestyle='-')
plt.axhline(y=0.75, color='red', linestyle='-')
plt.show()